## Linear Programming for Multi grade wheat contracts

#### Code is optimizing Profarmer eg correctly.

#### Now work on adding all grades, more contracts and use pandas dataframe

#### code idea came from sausage blending problem

http://benalexkeen.com/linear-programming-with-python-and-pulp-part-4/

In [14]:
from pulp import *

In [15]:
# Instantiate our problem class
model = pulp.LpProblem("Profit maximise wheat allocation", pulp.LpMaximize)

In [16]:
# construct our dicision varable lists

contract_types = ['contract_1', 'contract_2', 'cash']

grades = ['ASW', 'APW', 'H1']


In [17]:
# Each of these decision variables will have similar characteristics (lower bound of 0, continuous variables). 
# Therefore we can use PuLP’s LpVariable object’s dict functionality, we can provide our tuple indices.
# These tuples will be keys for the weight_tonnes dict of decision variables

grade_weight = pulp.LpVariable.dicts("weight tonnes",
                                     ((i, j) for i in contract_types for j in grades),
                                     lowBound=0,
                                     cat='Integer')

In [18]:
grade_weight

{('contract_1', 'ASW'): weight_tonnes_('contract_1',_'ASW'),
 ('contract_1', 'APW'): weight_tonnes_('contract_1',_'APW'),
 ('contract_1', 'H1'): weight_tonnes_('contract_1',_'H1'),
 ('contract_2', 'ASW'): weight_tonnes_('contract_2',_'ASW'),
 ('contract_2', 'APW'): weight_tonnes_('contract_2',_'APW'),
 ('contract_2', 'H1'): weight_tonnes_('contract_2',_'H1'),
 ('cash', 'ASW'): weight_tonnes_('cash',_'ASW'),
 ('cash', 'APW'): weight_tonnes_('cash',_'APW'),
 ('cash', 'H1'): weight_tonnes_('cash',_'H1')}

In [19]:
# A dictionary of prices in contract 1

contract_1 = {'ASW':247,
             'APW':267,
             'H1':277}

# A dictionary of prices in contract 2

contract_2 = {'ASW':199,
             'APW':224,
             'H1':234}

# A dictionary of prices in contract 3

cash = {'ASW':206,
             'APW':222,
             'H1':252}

contract_dict = {'C1':contract_1,
                'C2':contract_2,
                'C3':cash}


In [20]:
# Objective Function is added to 'model' first

# need to work on below code to simplify ie make more pythonic

list_a = [i for k,v in contract_dict.items() for i in v.values()]
list_b = [l for l in (list(grade_weight.values()))]

model += (
    pulp.lpSum([a*b for a,b in zip(list_a,list_b)])), "Total profit from selling all wheat"


In [21]:
# constraints
model += pulp.lpSum([grade_weight['contract_1', j] for j in grades]) == 500, "contract 1 requirement"
model += pulp.lpSum([grade_weight['contract_2', j] for j in grades]) == 500, "contract 2 requirement"
model += pulp.lpSum([grade_weight['cash', j] for j in grades]) == 1000, "cash requirement"
model += pulp.lpSum([grade_weight[i, 'ASW'] for i in contract_types]) <= 500
model += pulp.lpSum([grade_weight[i, 'APW'] for i in contract_types]) <= 1000
model += pulp.lpSum([grade_weight[i, 'H1'] for i in contract_types]) <= 500

In [22]:
print(model)

Profit maximise wheat allocation:
MAXIMIZE
222*weight_tonnes_('cash',_'APW') + 206*weight_tonnes_('cash',_'ASW') + 252*weight_tonnes_('cash',_'H1') + 267*weight_tonnes_('contract_1',_'APW') + 247*weight_tonnes_('contract_1',_'ASW') + 277*weight_tonnes_('contract_1',_'H1') + 224*weight_tonnes_('contract_2',_'APW') + 199*weight_tonnes_('contract_2',_'ASW') + 234*weight_tonnes_('contract_2',_'H1') + 0
SUBJECT TO
contract_1_requirement: weight_tonnes_('contract_1',_'APW')
 + weight_tonnes_('contract_1',_'ASW') + weight_tonnes_('contract_1',_'H1')
 = 500

contract_2_requirement: weight_tonnes_('contract_2',_'APW')
 + weight_tonnes_('contract_2',_'ASW') + weight_tonnes_('contract_2',_'H1')
 = 500

cash_requirement: weight_tonnes_('cash',_'APW')
 + weight_tonnes_('cash',_'ASW') + weight_tonnes_('cash',_'H1') = 1000

_C1: weight_tonnes_('cash',_'ASW') + weight_tonnes_('contract_1',_'ASW')
 + weight_tonnes_('contract_2',_'ASW') <= 500

_C2: weight_tonnes_('cash',_'APW') + weight_tonnes_('contra

In [10]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [11]:
for var in grade_weight:
    var_value = grade_weight[var].varValue
    print("The weight of {0} in {1} wheat is {2} tonnes".format(var[1], var[0], var_value))

The weight of ASW in contract_1 wheat is 0.0 tonnes
The weight of APW in contract_1 wheat is 500.0 tonnes
The weight of H1 in contract_1 wheat is 0.0 tonnes
The weight of ASW in contract_2 wheat is 0.0 tonnes
The weight of APW in contract_2 wheat is 500.0 tonnes
The weight of H1 in contract_2 wheat is 0.0 tonnes
The weight of ASW in cash wheat is 500.0 tonnes
The weight of APW in cash wheat is 0.0 tonnes
The weight of H1 in cash wheat is 500.0 tonnes


In [12]:
total_profit = pulp.value(model.objective)

print("The total profit is ${} for contracts".format(round(total_profit, 0)))

The total profit is $474500.0 for contracts


In [23]:
list_a

[247, 267, 277, 199, 224, 234, 206, 222, 252]

In [24]:
list_b

[weight_tonnes_('contract_1',_'ASW'),
 weight_tonnes_('contract_1',_'APW'),
 weight_tonnes_('contract_1',_'H1'),
 weight_tonnes_('contract_2',_'ASW'),
 weight_tonnes_('contract_2',_'APW'),
 weight_tonnes_('contract_2',_'H1'),
 weight_tonnes_('cash',_'ASW'),
 weight_tonnes_('cash',_'APW'),
 weight_tonnes_('cash',_'H1')]

In [26]:
print(zip(list_a,list_b))